### Connect to Duckdb and Inspect Database Schema


In [84]:
import duckdb

with duckdb.connect("data/sakila.duckdb") as conn:
    desc = conn.sql("DESC;").df()
desc

,database,schema,name,column_names,column_types,temporary
0,sakila,staging,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,sakila,staging,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,sakila,staging,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,sakila,staging,actor,"[actor_id, first_name, last_name, last_update,...","[DECIMAL(38,9), VARCHAR, VARCHAR, TIMESTAMP, V...",False
4,sakila,staging,address,"[address_id, address, address2, district, city...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, VA...",False
5,sakila,staging,category,"[category_id, name, last_update, _dlt_load_id,...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
6,sakila,staging,city,"[city_id, city, country_id, last_update, _dlt_...","[BIGINT, VARCHAR, BIGINT, TIMESTAMP, VARCHAR, ...",False
7,sakila,staging,country,"[country_id, country, last_update, _dlt_load_i...","[BIGINT, VARCHAR, TIMESTAMP, VARCHAR, VARCHAR]",False
8,sakila,staging,customer,"[customer_id, store_id, first_name, last_name,...","[BIGINT, BIGINT, VARCHAR, VARCHAR, VARCHAR, BI...",False
9,sakila,staging,film,"[film_id, title, description, release_year, la...","[BIGINT, VARCHAR, VARCHAR, VARCHAR, BIGINT, BI...",False


### Load and Register Tables from Duckdb Staging

In [ ]:
la = {}
with duckdb.connect("data/sakila.duckdb") as conn:
    for name in desc["name"]:
        la[name] = conn.sql(f"FROM staging.{name};").df()
        duckdb.register(name, la[name])
        


: 

: 

## Task 1 - EDA in python

### a) Which movies are longer than 3 hours (180 minutes), show the title and its length?

In [ ]:
la['film'].head(3)

: 

: 

In [ ]:
duckdb.sql("SELECT title, length FROM film WHERE length > 180 ORDER BY length;").df()

: 

: 

### b) Which movies have the word "love" in its title? Show the following columns title, rating, length, description

In [ ]:
duckdb.sql("SELECT title, rating, length, description FROM film WHERE title ILIKE '%love%' ORDER BY length DESC;").df()

: 

: 

### c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and longest movie length

In [ ]:
duckdb.sql("SELECT MIN(length) AS min_length_minutes, ROUND(AVG(length)) AS avg_length_minutes, MEDIAN(length) AS mean_length_minutes, MAX(length) AS max_length_minutes FROM film; ").df()

: 

: 

 ### d) The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie.
 ### The Manager wants to know the 10 most expensive movies to rent per day.

In [ ]:
la["film"].head(1)

: 

: 

In [ ]:
duckdb.sql("SELECT title, rental_rate/rental_duration AS rate_per_day FROM film ORDER BY rate_per_day DESC LIMIT 10;").df()

: 

: 

### e) Which actors have played in most movies? Show the top 10 actors with the number of movies they have played in.

In [ ]:
duckdb.sql("SELECT  a.first_name || ' ' || a.last_name AS actor_name, COUNT( distinct fa.film_id) AS total_films FROM actor a LEFT JOIN film_actor fa ON a.actor_id = fa.actor_id GROUP BY  actor_name  ORDER BY total_films DESC LIMIT 10;").df()

: 

: 

### 1) Identify the titles of the top 3 films that have the highest total revenue  but were rented the least number of times overall .
### Show the film title, the total_revenue, and the rental_count.


In [ ]:
duckdb.sql("SELECT f.title, SUM(p.amount) AS total_revenue, COUNT(r.rental_id) AS rental_count FROM film f LEFT JOIN inventory i ON f.film_id = i.film_id LEFT JOIN rental r ON i.inventory_id = r.inventory_id LEFT JOIN payment p ON r.rental_id = p.rental_id GROUP BY f.title ORDER BY total_revenue DESC, rental_count ASC LIMIT 3;").df()

: 

: 

### 2) Identify the top 5 film categories that have the highest average replacement cost for their films,
### but currently have the lowest average number of copies in inventory across all stores.
### Show the category_name, the average_replacement_cost, and the average_inventory_count.


In [ ]:
duckdb.sql("SELECT c.name AS category_name, ROUND(AVG(f.replacement_cost)) AS avg_replacement_cost, ROUND(COUNT(i.inventory_id)/ COUNT(DISTINCT f.film_id)) AS avg_inventory_count FROM category c LEFT JOIN film_category fc ON c.category_id = fc.category_id LEFT JOIN film f ON fc.film_id = f.film_id LEFT JOIN inventory i ON f.film_id = i.film_id GROUP BY category_name ORDER BY avg_replacement_cost DESC, avg_inventory_count ASC LIMIT 5;").df()

: 

: 

### 3) Which customers return movies the fastest on average?

In [ ]:
la["rental"].head(3)

: 

: 

In [ ]:
duckdb.sql("DESC rental;").df()

: 

: 

In [ ]:
duckdb.sql("SELECT c.customer_id, c.first_name || ' ' || c.last_name AS customer_name, ROUND(AVG(date_diff('day', r.rental_date , r.return_date))) AS avg_rental_duration, COUNT(r.rental_id) AS total_rentals FROM customer c JOIN rental r ON c.customer_id = r.customer_id WHERE r.return_date IS NOT NULL GROUP BY c.customer_id, customer_name ORDER BY avg_rental_duration, customer_name LIMIT 5;").df()


: 

: 

### ASSIGNING TO A VARIABALE AND TRASFER TO "REFINED" SCHEMA 

In [ ]:
customer_rental_duration = duckdb.sql("SELECT c.customer_id, c.first_name || ' ' || c.last_name AS customer_name, ROUND(AVG(date_diff('day', r.rental_date , r.return_date))) AS avg_rental_duration, COUNT(r.rental_id) AS total_rentals FROM customer c JOIN rental r ON c.customer_id = r.customer_id WHERE r.return_date IS NOT NULL GROUP BY c.customer_id, customer_name ORDER BY avg_rental_duration, customer_name;").df()


: 

: 

In [ ]:
duckdb.sql("""
            CREATE SCHEMA IF NOT EXISTS refined;
           
           CREATE OR REPLACE TABLE refined.customer_rental_duration AS 
           SELECT * FROM customer_rental_duration;
""")

: 

: 

In [ ]:
duckdb.sql("DESCRIBE refined.customer_rental_duration").df()



: 

: 

### Bar chart with top 10 fastest returners

In [ ]:
import matplotlib.pyplot as plt

df_chart1 = customer_rental_duration.head(10)

fig, ax = plt.subplots(figsize=(6,6))
ax.bar(df_chart1['customer_name'], df_chart1['avg_rental_duration'], color='skyblue')
ax.set_xlabel('Customers')
ax.set_ylabel('Average Rental Duration (days)')
ax.set_title('Top 10 Fastest Returning Customers')
ax.tick_params(axis='x', rotation=65)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.savefig("customer_rental_duration_top10.png", dpi=100, bbox_inches='tight')

plt.tight_layout()
plt.show()



: 

: 

### 4) Which actors have the highest average rental duration for the movies they star in?

In [ ]:
duckdb.sql("SELECT (a.actor_id:: INT) AS actor_id, a.first_name || ' ' || a.last_name AS actor_name, COUNT(fa.film_id) AS total_films, ROUND(AVG(date_diff('day' , r.rental_date, r.return_date))) AS avg_rental_duration_days FROM actor a LEFT JOIN film_actor fa ON a.actor_id = fa.actor_id LEFT JOIN film f ON fa.film_id = f.film_id LEFT JOIN inventory i ON f.film_id = i.film_id LEFT JOIN rental r ON i.inventory_id = r.inventory_id GROUP BY a.actor_id, actor_name ORDER BY avg_rental_duration_days DESC, actor_name ASC LIMIT 5;").df()

: 

: 

### 5) Which film rating generates the most total rental days?

In [ ]:
duckdb.sql("SELECT f.rating, SUM(DATE_DIFF('day', r.rental_date , r.return_date)) AS total_days_rented FROM film f LEFT JOIN inventory i ON f.film_id = i.film_id LEFT JOIN rental r ON i.inventory_id = r.inventory_id GROUP BY f.rating ORDER BY total_days_rented DESC;").df()

: 

: 

### ASSIGNING TO A VARIABALE AND TRASFER TO "REFINED" SCHEMA 

In [ ]:
rating_total_days = duckdb.sql("SELECT f.rating, SUM(DATE_DIFF('day', r.rental_date , r.return_date)) AS total_days_rented FROM film f LEFT JOIN inventory i ON f.film_id = i.film_id LEFT JOIN rental r ON i.inventory_id = r.inventory_id GROUP BY f.rating ORDER BY total_days_rented DESC;").df()

: 

: 

In [ ]:
duckdb.sql("""
            CREATE OR REPLACE TABLE refined.rating_total_days AS 
           SELECT * FROM rating_total_days;
""")

: 

: 

### Pie chart with Total Rental Days By Film Rating 

In [ ]:
import matplotlib.pyplot as plt

sizes = rating_total_days["total_days_rented"].values
labels = rating_total_days["rating"].values
colors = plt.cm.Set3(range(len(labels))) 

plt.figure(figsize=(3,3))
plt.pie(
    sizes,
    labels=labels,
    autopct='%1.1f%%',
    startangle=140,
    colors=colors,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1}
)

plt.title('Total Rental Days by Film Rating')
plt.tight_layout()

plt.savefig("total_rental_days_pie.png", dpi=100)

plt.show()




: 

: 

## Task 2 - graphs

### a) Who are our top 5 customers by total spend? The Manager wants to know so that they can reward them
### with special offers. Create a bar chart showing the top 5 customers by total spend.

In [ ]:
duckdb.sql("SELECT c.customer_id, c.first_name || ' ' || c.last_name AS customer_name, SUM(p.amount) AS total_spend FROM customer c LEFT JOIN payment p ON c.customer_id = p.customer_id GROUP BY c.customer_id, customer_name ORDER BY total_spend DESC, customer_name ASC LIMIT 5" ).df()

: 

: 

### ASSIGNING TO A VARIABALE AND TRASFER TO "REFINED" SCHEMA 

In [ ]:
top_customers = duckdb.sql("SELECT c.customer_id, c.first_name || ' ' || c.last_name AS customer_name, SUM(p.amount) AS total_spend FROM customer c LEFT JOIN payment p ON c.customer_id = p.customer_id GROUP BY c.customer_id, customer_name ORDER BY total_spend DESC, customer_name ASC" ).df()

: 

: 

In [ ]:
duckdb.sql("""
            CREATE OR REPLACE TABLE refined.top_customers_total_spend AS 
           SELECT * FROM top_customers 
""")

: 

: 

### Horizontal Bar Chart for Top Five Customers

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

top5_customers = top_customers.sort_values(by="total_spend", ascending=False).head(5)

top5_customers_sorted = top5_customers.sort_values(by="total_spend", ascending=True)

names = top5_customers_sorted["customer_name"]
totals = top5_customers_sorted["total_spend"]

plt.figure(figsize=(6,4))
ax = plt.gca()

num_bars = len(names)
colors = cm.Blues(np.linspace(0.4, 0.8, num_bars))  

bars = ax.barh(names, totals, color=colors)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel("Total Spend")
plt.title("Top 5 Customers by Total Spend")
plt.tight_layout()

plt.savefig("top_5_customers.png", dpi=100, bbox_inches='tight')
plt.show()




: 

: 

### b) How much money does each film category bring in? Make a bar chart showing total revenue per film category.

In [ ]:
duckdb.sql("SELECT c.name AS category_name, SUM(p.amount) AS total_revenue FROM category c LEFT JOIN film_category fc ON c.category_id = fc.category_id LEFT JOIN film f ON fc.film_id = f.film_id LEFT JOIN inventory i ON f.film_id = i.film_id LEFT JOIN rental r ON i.inventory_id = r.inventory_id LEFT JOIN payment p ON r.rental_id = p.rental_id GROUP BY category_name ORDER BY total_revenue DESC;").df()

: 

: 

### ASSIGNING TO A VARIABALE AND TRASFER TO "REFINED" SCHEMA 

In [ ]:
category_total_revenue = duckdb.sql("SELECT c.name AS category_name, SUM(p.amount) AS total_revenue FROM category c LEFT JOIN film_category fc ON c.category_id = fc.category_id LEFT JOIN film f ON fc.film_id = f.film_id LEFT JOIN inventory i ON f.film_id = i.film_id LEFT JOIN rental r ON i.inventory_id = r.inventory_id LEFT JOIN payment p ON r.rental_id = p.rental_id GROUP BY category_name ORDER BY total_revenue DESC;").df()

: 

: 

In [ ]:
duckdb.sql("""
            CREATE OR REPLACE TABLE refined.category_revenue AS SELECT *
           FROM category_total_revenue

""")

: 

: 

### Bar chart for Category Revenue

In [ ]:
import duckdb
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

df = duckdb.sql("SELECT * FROM refined.category_revenue").df()

norm = (df["total_revenue"] - df["total_revenue"].min()) / (df["total_revenue"].max() - df["total_revenue"].min())
norm = 0.3 + 0.7 * norm  # scale to [0.3, 1]

colors = cm.Greens(norm)

plt.figure(figsize=(10,6))
plt.bar(df["category_name"], df["total_revenue"], color=colors)
plt.xlabel("Film Category")
plt.ylabel("Total Revenue")
plt.title("Total Revenue per Film Category")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("category_revenue_chart.png", dpi=300)
plt.show()



: 

: 

In [ ]:
duckdb.sql("SHOW TABLES FROM refined;")

: 

: 

In [ ]:
duckdb.sql("DESCRIBE refined.category_revenue;")

: 

: 